In [30]:
fraud_data_path = "/kaggle/input/frauddata/EngineeredTransactionDataset.csv"

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [35]:
engineered_df = pd.read_csv(fraud_data_path)

In [36]:
engineered_df = pd.concat([engineered_df, 1*pd.get_dummies(engineered_df['type'])], axis=1).drop('type', axis=1)
engineered_df = engineered_df.drop(['PAYMENT'], axis=1)

In [37]:
engineered_df.drop("Unnamed: 0", axis=1, inplace=True)

In [38]:
engineered_df

,step,amount,oldbalanceOrig,oldbalanceDest,isFraud,trans_frequency_Dest,avg_trans_amount_Dest,variability_trans_amount_Dest,big_trans_flag,zero_new_balance_Dest_flag,zero_new_balance_Orig_flag,CASH_IN,CASH_OUT,DEBIT,TRANSFER
0,1,9839.64,170136.00,0.00,0,1,9.839640e+03,0.000000e+00,0,1,0,0,0,0,0
1,1,1864.28,21249.00,0.00,0,1,1.864280e+03,0.000000e+00,0,1,0,0,0,0,0
2,1,181.00,181.00,0.00,1,44,2.255178e+05,2.526558e+05,0,1,1,0,0,0,1
3,1,181.00,181.00,21182.00,1,41,4.394442e+05,8.789905e+05,0,1,1,0,1,0,0
4,1,11668.14,41554.00,0.00,0,1,1.166814e+04,0.000000e+00,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,1,2,3.323914e+05,1.031071e+04,0,0,1,0,1,0,0
6362616,743,6311409.28,6311409.28,0.00,1,1,6.311409e+06,0.000000e+00,1,1,1,0,0,0,1
6362617,743,6311409.28,6311409.28,68488.84,1,3,2.212471e+06,3.551043e+06,1,0,1,0,1,0,0
6362618,743,850002.52,850002.52,0.00,1,1,8.500025e+05,0.000000e+00,1,1,1,0,0,0,1


In [ ]:
!pip install catboost

In [63]:
from catboost import Pool, CatBoostRegressor, CatBoostClassifier, cv
from catboost.utils import get_roc_curve

In [39]:


X = engineered_df.drop('isFraud', axis = 1)
y = engineered_df.isFraud


In [40]:
X

,step,amount,oldbalanceOrig,oldbalanceDest,trans_frequency_Dest,avg_trans_amount_Dest,variability_trans_amount_Dest,big_trans_flag,zero_new_balance_Dest_flag,zero_new_balance_Orig_flag,CASH_IN,CASH_OUT,DEBIT,TRANSFER
0,1,9839.64,170136.00,0.00,1,9.839640e+03,0.000000e+00,0,1,0,0,0,0,0
1,1,1864.28,21249.00,0.00,1,1.864280e+03,0.000000e+00,0,1,0,0,0,0,0
2,1,181.00,181.00,0.00,44,2.255178e+05,2.526558e+05,0,1,1,0,0,0,1
3,1,181.00,181.00,21182.00,41,4.394442e+05,8.789905e+05,0,1,1,0,1,0,0
4,1,11668.14,41554.00,0.00,1,1.166814e+04,0.000000e+00,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,2,3.323914e+05,1.031071e+04,0,0,1,0,1,0,0
6362616,743,6311409.28,6311409.28,0.00,1,6.311409e+06,0.000000e+00,1,1,1,0,0,0,1
6362617,743,6311409.28,6311409.28,68488.84,3,2.212471e+06,3.551043e+06,1,0,1,0,1,0,0
6362618,743,850002.52,850002.52,0.00,1,8.500025e+05,0.000000e+00,1,1,1,0,0,0,1


In [41]:
X_not_test, X_test, y_not_test, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42, stratify = y)
X_train, X_eval, y_train, y_eval = train_test_split(X_not_test, y_not_test, test_size = 0.2, stratify = y_not_test, random_state = 42)

In [ ]:
# Training optimized model

best_model = CatBoostClassifier(iterations=2000,
                           depth = 8,
                           learning_rate=0.1,
                           loss_function='Logloss',
                           eval_metric = 'Recall')

best_model.fit(X_train, y_train,
          eval_set = (X_eval, y_eval),
          verbose=100,
          early_stopping_rounds = 100)

In [ ]:
print(f'Precision on test: {precision_score(best_model.predict(X_test), y_test)}\n'
f'Recall on test: {recall_score(best_model.predict(X_test), y_test)}\n'
f'F1 on test {f1_score(best_model.predict(X_test), y_test)}\n'
f'ROC AUC on test: {roc_auc_score(best_model.predict(X_test), y_test)}')

In [ ]:
test_predictions = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

# Classification report
report = classification_report(y_test.values, test_predictions)
print("Classification Report:")
print(report)

In [ ]:
test_predictions

In [ ]:
print(f'Precision on test: {precision_score(gbm_model.predict(X_test), y_test)}\n'
f'Recall on test: {recall_score(gbm_model.predict(X_test), y_test)}\n'
f'F1 on test {f1_score(gbm_model.predict(X_test), y_test)}\n'
f'ROC AUC on test: {roc_auc_score(gbm_model.predict(X_test), y_test)}')

In [15]:
test_features = xgb_model.predict(X_test)

In [16]:
from sklearn.metrics import classification_report

# Classification report
report = classification_report(y_test.values, test_features)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635441
           1       0.96      0.85      0.90       821

    accuracy                           1.00    636262
   macro avg       0.98      0.92      0.95    636262
weighted avg       1.00      1.00      1.00    636262



In [17]:
xgb_model

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2000,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [19]:
import xgboost as xgb

# Assuming xgb_regressor is your trained XGBoost regressor
xgb_model.save_model('xgb_model.json')

In [20]:
loaded_xgb_model = xgb.XGBClassifier()

In [21]:
loaded_xgb_model.load_model('/kaggle/working/xgb_model.json')

In [22]:
test_predictions = xgb_model.predict(X_test)

In [23]:
from sklearn.metrics import classification_report

# Classification report
report = classification_report(y_test.values, test_predictions)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635441
           1       0.96      0.85      0.90       821

    accuracy                           1.00    636262
   macro avg       0.98      0.92      0.95    636262
weighted avg       1.00      1.00      1.00    636262



In [42]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [43]:
# Create LightGBM Datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [44]:
# Set parameters
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Evaluation metric
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 2,
    'early_stopping_rounds': 30  # Parameter for early stopping
}

# Train the model
num_round = 2000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions
predictions = bst.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions.round())
auc = roc_auc_score(y_test, predictions)

print("Accuracy:", accuracy)
print("AUC:", auc)

[LightGBM] [Info] Number of positive: 5914, number of negative: 4575172
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.759013
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.308546
[LightGBM] [Debug] init for col-wise cost 0.121735 seconds, init for row-wise cost 0.569873 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.265826 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1640
[LightGBM] [Info] Number of data points in the train set: 4581086, number of used features: 14
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001291 -> initscore=-6.651077
[LightGBM] [Info] Start training from score -6.651077
[LightGBM] [Debug] Re-bagging, using 3666362 data to train
[LightGBM] [Debug] Trained a tree w

In [45]:
test_predictions = bst.predict(X_test)

In [46]:
true_predictions = test_predictions > 0.5

In [47]:
from sklearn.metrics import classification_report

# Classification report
report = classification_report(y_test.values, true_predictions)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635441
           1       0.96      0.84      0.89       821

    accuracy                           1.00    636262
   macro avg       0.98      0.92      0.95    636262
weighted avg       1.00      1.00      1.00    636262



In [49]:
# Save the trained model to a file
bst.save_model('lightgbm_model.txt')